In [1]:
#make sure you install everyhings

from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


pd.set_option('display.max_rows', None)
# firefox or Chrome|


from IPython.display import display, HTML

from textblob_de import TextBlobDE


#### get the page links (need to change the Input)

In [2]:
%%time
# get the pag links
htmls = []

for page in range(1,380):  # change 165 to the 'total pages +1', e.g. the total pages are 33, you need to insert 33+1 = '34'
    
    url = 'https://www.hagel-shop.de/hautpflege.html?p={}'  # insert the new link with changing the 200 to {}!!!!
    htmls.append(url.format(page))
    
    

len(htmls)

CPU times: total: 0 ns
Wall time: 0 ns


379

#### get the product links.  

In [3]:
%%time
# get the product links

productlinks = []

for html in htmls:
    
    #page = 1
    r = requests.get(html)
    soup = BeautifulSoup(r.content, 'lxml')
    productlist = soup.find_all('li', class_='item')
    
    for item in productlist:
        for link in item.find_all('a', class_='product-image', href=True, limit=1):
           productlinks.append(link['href'])
    
    #page = page + 1
len(productlinks)


CPU times: total: 1min 29s
Wall time: 14min 52s


7561

#### basic setup. 

In [4]:
############## soup

def brand(soup):
    try:
        brand = soup.find('span', itemprop="brand").text.replace('-','').replace("%", ".")
    except AttributeError: 
        return 
    
    return brand


def name1(soup):
    try:
        name = soup.find('h1', itemprop="name").text.replace('\n', ',')
    except AttributeError: 
        return 
    
    return name

def reviews(soup):
    try:
        reviews = soup.find('span', itemprop="ratingCount").text.replace('\n', '').replace(' ', '')
    except AttributeError:
        return 0
    
    return reviews


def ratings(soup):
    try:
        ratings = soup.find('span', itemprop="ratingValue").text.replace('\n', '').replace(' ', '')
    except AttributeError:
        return 0
    
    return ratings


def discount(soup):
    try:
        discount = soup.find('div', class_="product-label percent").text.replace('-','').replace("%", ".")
    except AttributeError:
        return 
    
    return discount

def EAN(soup):
    try:
        EAN = soup.find('meta', itemprop="gtin13")['content']
    except (AttributeError,TypeError):
        return 
    
    return EAN


def comments(soup):
    try:
        comments = ''.join([j.text.replace('\n','') for j in soup.find('ol',class_="reviews-list").find_all('p')])
    except AttributeError: 
        return 
    
    return comments 

def check(soup):
    try:
        check = soup.find('span', class_="text").text.split(' ')[1]
    except IndexError: 
        check = 'Nope'
    
    return check

############### row

def priceOne(row):
    try:
       # price = row.find('p', class_="special-price").text.replace('Jetzt', '').replace('€', '').replace('\n','').replace(' ','').replace(",", ".")
        price1 = row.find('p', class_='special-price').text.replace('Jetzt', '').replace('€', '').replace('\n','').replace(' ','').replace(",", ".")
    except AttributeError:
        price1 = row.find('div', class_='price-box').text.replace('Jetzt', '').replace('€', '').replace('\n','').replace(' ','').replace(",", ".")
    
    return price1


def priceTwo(row):
    try:
       # price = row.find('p', class_="special-price").text.replace('Jetzt', '').replace('€', '').replace('\n','').replace(' ','').replace(",", ".")
        price2 = row.find('p', class_='old-price').text.replace('€', '').replace('\n','').replace(' ','').replace(",", ".").replace('RegulärerPreis:', '')
    except AttributeError:
        price2 = priceOne(row)
    
    return price2


def size1(row):
    try:
        size = row.find('span', class_="text").text.replace('ml', '')
    except AttributeError:
        return
    
    return size


def identity(row):
    try:
        id = row.find('span', class_ = 'sku').text.replace('\n','').replace(' ','').replace('Art.Nr.','')
    except (AttributeError, TypeError):
        return
    
    return id

def deliveryTime1(row):
    try: 
        time = row.find('td',class_='item-stock-wrapper').find('span',class_='text').text.replace('Werktage', '').replace('Lieferzeit', '').replace(' ', '').split('-')[1]
    except AttributeError:
        return
    
    return time

#### start to Scrap.  

In [10]:
%%time

df  = pd.DataFrame(columns=['id','EAN','brand', 'name', 'discount','price1','price2', 'size',"reviews","ratings",'sorting','links','deliveryTime','comment'])


for i, link in enumerate(productlinks):

     r = requests.get(link)
     soup = BeautifulSoup(r.content, 'lxml') 
    
    
    #try: 
            #brands = soup.find('span', itemprop="brand").text.replace('-','').replace("%", ".")
     #except AttributeError:
        #brands = None
    
     #if brands in ['Kerastase','Olaplex','Alcina','KMS',"L'OREAL",'Paul Mitchell','Wella','Redken','Wella SP',"L'Oréal", 'Wella Deluxe']:

    

     if bool(len(soup.find('div',class_='product-essential').find_all('tr', class_='item')) == 0) is True:    


        try:
            price1 = soup.find('div', class_="price-details").find('span',itemprop="price").text.replace(",", ".")
        except AttributeError:
            price1 = None

        try:
            price2 = soup.find('p', class_="old-price").find('span',class_='price').text.replace('€', '').replace('\n','').replace(' ','').replace(",", ".")#.replace('RegulärerPreis:', '')
        except AttributeError:
            price2 = price1
        size = re.findall(".{6}ml$|.{6}g$",name1(soup))
        size = ','.join(size) # covert list to string

        try:
            size = size.split(' ')[-2]
        except IndexError:
            size = None

        id = soup.find('span', itemprop="sku").text
        try:
            deliveryTime = soup.find('div',class_ ='availability in-stock').find('span',class_='text').text.replace('Werktage', '').replace('Lieferzeit', '').replace(' ', '').split('-')[1]
        except AttributeError:
            deliveryTime = 2


        product = {

            'id': id ,
            'EAN': EAN(soup),
            'brand': brand(soup),
            'name': name1(soup),
            'discount': discount(soup),
            'price1': price1,
            'price2': price2,
            'size': size,
            "reviews": reviews(soup),
            "ratings": ratings(soup),
            'sorting': i,
            'links' : link,
            'deliveryTime':deliveryTime,
            'comments': comments(soup)

            }


        df = df.append(product, ignore_index=True) 



     elif check(soup) in ['ml','g']:

            for row in soup.find('div',class_='product-essential').find_all('tr', class_='item'):

                product = {


                'id': [identity(row)],
                'EAN': [EAN(soup)],
                'brand': [brand(soup)],
                'name': [name1(soup)],
                'discount': [discount(soup)],
                'price1': [priceOne(row)],
                'price2': [priceTwo(row)],
                'size': [size1(row)],
                "reviews": [reviews(soup)],
                "ratings": [ratings(soup)],
                'sorting': [i],
                'links' : [link],
                'deliveryTime':[deliveryTime1(row)],
                'comments': [comments(soup)]

                }

                product = pd.DataFrame(product)

                #df.append(product)
                df = df.append(product)


            df.append(df)
            df = df.reset_index().drop('index',  axis=1)


     else:

            color_in_name = soup.find('span', class_="text").text
            color_in_name = color_in_name.title().split()
            for row in soup.find('div',class_='product-essential').find_all('tr', class_='item'):

                color = row.find('span', class_="text").text
                inital_name = soup.find('h1', itemprop="name").text.replace('\n', ',')

                color = color.title().split()
                inital_name = inital_name.title().split()


                if any(x in inital_name for x in color_in_name):
                    for i in color_in_name:
                        try:
                            inital_name.remove(i) 
                        except ValueError:
                             inital_name
                        name = inital_name + color
                else:
                    name = inital_name+color



                product = {


                'id': [identity(row)],
                'EAN': [EAN(soup)],
                'brand': [brand(soup)],
                'name': ' '.join(name),
                'discount': [discount(soup)],
                'price1': [priceOne(row)],
                'price2': [priceTwo(row)],
                'size': link.split("-")[-2],
                "reviews": [reviews(soup)],
                "ratings": [ratings(soup)],
                'sorting': [i],
                'links' : [link],
                'deliveryTime':[deliveryTime1(row)],
                'comments': [comments(soup)]

                }

                product = pd.DataFrame(product)

                #df.append(product)
                df = df.append(product)


            df.append(df)
            df = df.reset_index().drop('index',  axis=1)
                      
df.to_excel('raw.xlsx', index=False)  
df.shape
    

CPU times: total: 18min 15s
Wall time: 2h 28min 22s


(8009, 14)

In [ ]:
display(HTML("<div style='height: 500px'>" + df.style.render() + "</div>"))

## Data Cleaning

In [64]:
%%time

# remove sizes from names

for i in range(0,df.shape[0]):
    df.at[i,'size'] = str(df['size'].iloc[i]).replace(' ','').replace(',','.').replace('g','')
    
for j in range(0,df.shape[0]):
    
    if df['size'].iloc[j] != None:
          size_ml = str(df['size'].iloc[j]) + ' ' + 'ml'
          size_g = str(df['size'].iloc[j]) + ' ' + 'g'

        
    if df.iloc[j]['size']  in df.iloc[j]['name'].split():
        df.at[j,'name'] = re.sub(size_ml,  '', df['name'].iloc[j])
        df.at[j,'name'] = re.sub(size_g,  '', df['name'].iloc[j])
    else: 
        for i in range(0,df.shape[0]):

            if df['size'].iloc[i] != None:
                  size_ml = str(df['size'].iloc[i]) + ' ' + 'ml'
                  size_g = str(df['size'].iloc[i]) + ' ' + 'g'

            if df.iloc[i]['size']  in df.iloc[j]['name'].split():
                    df.at[j,'name'] = re.sub(size_ml,  '', df['name'].iloc[j])
                    df.at[j,'name'] = re.sub(size_g,  '', df['name'].iloc[j])
                    break


df.to_excel('raw1.xlsx', index=False) 

CPU times: total: 30min 19s
Wall time: 30min 23s


In [8]:

#Remove brands from names

for i in range(0,df.shape[0]):
    df.at[i,'size'] = str(df['size'].iloc[i]).replace(' ','').replace(',','.').replace('g','')
    df.at[i,'name'] = re.sub(str(df.iloc[i]['brand']).title(),  '', str(df.iloc[i]['name']).title())

# reset the sorting/ ranking  
df.at[0,'sorting'] = 0
for i in range(1,df.shape[0]):
    
    if df['name'].iloc[i] == df['name'].iloc[i-1]:
        df.at[i,'sorting'] = df['sorting'].iloc[i-1]
    else:
        df.at[i,'sorting'] = df['sorting'].iloc[i-1] +1
        
# convert the variables below to float
df[["price1","price2",'ratings','reviews','sorting','discount']] = df[["price1","price2",'ratings','reviews','sorting','discount']].astype(float)


# convert the sizes to float
for i in range(0,df.shape[0]):
    try:
        df.at[i,'size'] = pd.to_numeric(df.loc[i]['size'], errors='ignore', downcast='float')
    except ValueError:
        pass

#If price2 is empty, then price2 = price1
for i in range(0, df.shape[0]):
    if pd.isnull(df.loc[i]['price2']):
        df.at[i,'price2'] = df.loc[i]['price1']



df['price2 minus price1'] = df['price2'] - df['price1']


#price per 100 ml
df['price1 per 100ml'] = None
df['price2 per 100ml'] = None

for i in range(0,df.shape[0]):
    try:
        df.at[i, 'price1 per 100ml']  = (df['price1'].iloc[i]/df['size'].iloc[i])*100
    except TypeError:    
        df.at[i,'price1 per 100ml'] = None

    try:
        df.at[i, 'price2 per 100ml']  = (df['price2'].iloc[i]/df['size'].iloc[i])*100
    except TypeError:    
        df.at[i,'price2 per 100ml'] = None
        

# create a new variable 'brand + name' 
df['brand + name'] = df['brand'] + ' ' + df['name'] 
for i in range(0,df.shape[0]):
    df.at[i,'brand + name'] = str(df.iloc[i]['brand + name']).title().replace('  ',' ')
        

#### Sentiment Analysis by Textblod

#### add category columns

In [17]:
### need to change

df['p_category'] = None # Change
df['pp_category'] = 'Hautpflege' # Change
df['main_category'] = 'Hautpflege' # Change
df['source'] = 'hagel' #Change

df.shape

(8009, 21)

In [22]:
#f = df[['id', 'EAN', 'brand', 'name', 'size', 'discount', 'price1', 'price2', 'price2 minus price1', 'price1 per 100ml','price2 per 100ml','reviews', 'ratings', 'sorting', 'p_category', 'pp_category', 'main_category', 'sourse', 'links', 'comments', 'polarity','scale polarity (reviews*polarity)','deliveryTime']]
#df = df[['id', 'EAN', 'brand', 'name', 'brand + name', 'size', 'discount', 'price1', 'price2', 'price2 minus price1', 'price1 per 100ml','price2 per 100ml','reviews', 'ratings', 'sorting', 'p_category', 'pp_category', 'main_category', 'source', 'links', 'deliveryTime']]

#df = df[['id', 'EAN', 'brand', 'name', 'brand + name','size', 'discount', 'price1', 'price2', 'price2 minus price1', 'price1 per 100ml', 'price2 per 100ml', 'reviews', 'ratings',  'sorting', 'p_category', 'pp_category', 'main_category','source' ,'links', 'deliveryTime']]

In [23]:
df.to_excel('Hautpflege.xlsx', index=False)  #change the name. but the format is '.xlsx'

df1 = pd.read_excel('Hautpflege.xlsx')  #change the name. but the format is '.xlsx'

df1.to_csv('Hautpflege.csv', index=False) #change the name. but the format is '.csv'!!!!


#### combine all over categories you have scrapped!

In [44]:



a = pd.read_csv("Farbe1.csv")  # Change
b = pd.read_csv("Strähnen, Haarkreide & Make-Up.csv")  # Change
c= pd.read_csv("Blondierung.csv")# Change
d= pd.read_csv("Tönung.csv")# Change
e= pd.read_csv("Zubehör.csv")# Change
#f = pd.read_csv("Reisegrößen.csv")
#haarpflegeset = pd.read_csv("haarpflegeset_flaconi.csv")
#haarserum = pd.read_csv("haarserum.csv")
#pflege = pd.read_csv("leave-in-pflege_flaconi.csv")
#haarmaske = pd.read_csv("haarmaske_flaconi.csv")
# Change



df = pd.concat([a, b, c,d,e])# Change
df = df.reset_index().drop('index',  axis=1)
df.shape

(1315, 18)